# GPIO

## ESPDuino32

De ESP32 heeft redelijk wat GPIO (General Purpose Input Output) pinnen. Sommige pinnen ondersteunen enkel één richting, sommige zijn strapping pins, nog andere hebben alternatieve functies, ...  Op de website van [RandomNerdTurtorials](https://randomnerdtutorials.com/esp32-pinout-reference-gpios/) staat hierover veel informatie, en een heel handig overzicht in tabelvorm. De belangrijkste zaken zijn hieronder overgenomen:

```{figure} ./images/esp32_pinout.png
:width: 500px
:align: left
:figwidth: image
:figclass: myBlockImg

ESP32 en mogelijkheden per GPIO.
```

### Input only

De pinnen 34, 35, 36 en 39 zijn enkel ingangen. Deze kunnen niet gebruikt worden als uitgang. Er zal echter geen fout optreden indien we dit proberen. Er zal simpelweg niets veranderen aan de logische toestand van de pin. Deze pinnen bevatten eveneens geen pull-up of pull-down weerstanden. Indien je dit nodig hebt zul je deze zelf extern moeten toevoegen.

### Capacitive touch

De pinnen 0, 2, 4, 12, 13, 14, 15, 27, 22 en 33 (10 in totaal) hebben de mogelijkheid om elektrostatische lading te detecteren. Indien een lange kabel, een stukje metaal of gelijk welk geleidend materiaal wordt aangebracht aan deze pin, kan het aanraken met de hand van deze de lading veranderen. Dit kan vervolgens gedetecteerd worden in de ESP. Opgelet, veel van deze pinnen zijn _strapping pins_. Zie hiervoor verder.

### Analog to Digital (ADC)

Er zijn in totaal 18 pinnen voor ADC beschikbaar, en dit verdeeld over twee banken (ADC1 en ADC2). In principe zitten er dus slecht twee ADC convertoren in de ESP, die vooraf worden gegaan door een multiplexer. De ADC1 heeft een multiplexer met 8 beschikbare ingangen, de ADC2 een multiplexer met 10 beschikbare ingangen. De voorkeur moet gegeven worden aan ADC1, aangezien enkel deze beschikbaar is wanneer WiFi wordt gebruikt. Voor IoT toepassingen zal WiFi altijd nodig zijn, dus is de keuze voor de ADC1 aangewezen. Indien men toch meer analoge ingangen nodig heeft zal de WiFi (tijdelijk) moeten gedeactiveerd worden.

* ADC1: 32, 33, 34, 35, 36, 37, 38, 39
* ADC2: 0, 2, 4, 12, 13, 14, 15, 25, 26, 27 (veel van deze stemmen overeen met de capacitive touch)

### Digital to Analog (DAC)

Voor de omgekeerde weg zijn er slechts twee mogelijkheden (zonder demultiplexer), namelijk DAC1 (pin 25) en DAC2  (pin 26). Herrouteren van deze signalen is moeilijk, dus deze pinnen moeten hiervoor altijd gebruikt worden.

### PWM

De ESP32 heeft intern 16 PWM controllers, waarbij iedere uitgangspin kan gebruikt worden om één van de 16 controller uitgangen mee te verbinden.

### Strapping pins

Iets die speciaal/eigenaardig is aan de ESP zijn de _strapping pins_. Dit zijn pinnen die tijdens het booten op logisch niveau worden ingelezen. Door een specifiek logisch niveau aan te leggen op deze pinnen wordt de ESP in een bepaalde _boot mode_ gebracht. _De belangrijkste boot mode voor ons is 0x13 (SPI_FAST_FLASH_BOOT), die het programma laadt vanuit flash geheugen. Het "programma" die voor ons interessant is, is MicroPython._

```{figure} ./images/bootmode.png
:width: 500px
:align: left
:figwidth: image
:figclass: myBlockImg

Indicatie van bootmode op de CLI tijdens booten van de ESP32.
```

Mocht MicroPython niet starten zoals verwacht, dan kan in de CLI gecontroleerd worden wat fout loopt. Meestal zal de _boot mode_ afwijken van de verwachtte 0x13, en dit wellicht te wijten aan een elektrische verbindingen aangebracht op de _strapping pins_.

Volgende pinnen met hun overeenkomstige boot waarde bestaan:
* 0x01: GPIO5
* 0x02: GPIO15
* 0x04: GPIO4
* 0x08: GPIO2
* 0x10: GPIO0
* 0x20: GPIO12

Om 0x13 te bekomen hebben we nood aan 0x10 + 0x02 + 0x01. Dit betekent dat GPIO0, GPIO15 en GPIO5 tijdens het booten logisch 1 moeten zijn (gestrapped naar de voeding), en alle andere pinnen logisch 0 (gestrapped naar de massa). Mocht er per ongeluk een van de pinnen foutief hoog of laag gemaakt zijn tijdens het booten, dan zal een andere boot mode getoond worden. Via bovenstaand overzicht zou het probleem snel achterhaald moeten kunnen worden.

## Inputs

Om gebruik te kunnen maken van de GPIO functionaliteiten in MicroPython moeten we allereerst de klasse `Pin` importeren vanuit de `machine` klasse. Deze klasse omvat alle functionaliteiten die een standaard GPIO aanbiedt (dus basis in- en uitgang). Dit is identiek voor zowel een in- als een uitgang op de ESP.

```python
from machine import Pin
```

### Digitaal

Een toepasselijk voorbeeld voor een digitale input is het binnenlezen van de status van een drukknop. Deze drukknop kan (elektrisch) op twee verschillende manieren aangesloten zijn op de hardware:

```{figure} ./images/digital_input.png
:width: 500px
:align: left
:figwidth: image
:figclass: myBlockImg

Mogelijkheden tot aansluiten van een drukknop.
```
Bovenstaande methoden zullen beiden ongeveer identiek werken, m.u.v. het ingelezen logische niveau tijdens bediening. De rechtste implementatie zal een logische 1 genereren wanneer ingedrukt, terwijl de linkse implementatie een logische 0 zal generen. De (optionele) weerstand R1 is hierbij een _pullup_ of _pulldown_ weerstand. De (optionele) weerstand R2 een stroombeveiliging, en dit wanneer de eindgebruiker foutief een ingang als uitgang zou definiëren. _In beide gevallen wordt gemeld dat de weerstand optioneel is. Men kan opteren om beide weg te laten indien de code correct wordt geschreven. In uiteindelijke implementaties zorgen minder componenten voor grotere [MTBF](https://nl.wikipedia.org/wiki/Mean_time_between_failures) cijfers, wat natuurlijk altijd een meerwaarde is voor het product._

Als eerst moeten we een object aanmaken van de klasse `Pin` die onze digitale ingang voorstelt. Dit kan als volgt:

```python
GPIO26 = Pin(26, Pin.IN)
```

Voor bijna alle pinnen, m.u.v. de pinnen 34, 35, 36 & 39, kan er geopteerd worden om een pullup of pulldown weerstand intern in de ESP te activeren, zodat de externe weerstand kan weggelaten worden. Dit kan door de constructor van de `Pin` uit te breiden als volgt:

```python
GPIO26 = Pin(26, Pin.IN, Pin.PULL_UP) #Pin.PULL_DOWN bestaat eveneens
```

Het binnenlezen van de logische waarde kan vervolgens via de methode `value()`:

```python
print(GPIO26.value())
```

### Interrupts

De voortdurende controle of een ingang al dan niet van logische waarde verandert vraagt redelijk wat CPU tijd. Deze methode wordt in het vakjargon _polling_ genaamd, waarbij de status van de ingangspin in de `main()` voortdurend wordt afgevraagd (_gepolled_). Dit kan voorkomen worden door aan de ingangspin een callback (IRQ, Interrupt ReQuest) te koppelen die wordt opgeroepen wanneer de toestand van de `Pin` wijzigt. 

```python
GPIO26.irq(<callback functie>)
```

Volgende implementatie toont de waarde van de pin wanneer deze wijzigt van toestand:

```python
from machine import Pin

GPIO26 = Pin(26, Pin.IN, Pin.PULL_UP)

def button_isr(context):
    print(context.value())

def main():
    GPIO26.irq(button_isr)
    while(True):
        pass

if(__name__=="__main__"):
    main()
```

Bovenstaande implementatie kan nog uitgebreid worden. Volgende zaken zijn nog van belang bij de `irq` methode:
* `trigger` laat toe om de `callback` slechts op te roepen bij een overgang van/naar een bepaald logisch niveau. via `Pin.IRQ_FALLING` en/of `Pin.IRQ_RISING` kan er geselecteerd worden op een _rising_ of _falling_ edge.
* `priority` laat toe een prioriteit te geven aan de interrupt. Hoe hoger het getal, hoe groter de prioriteit. 

Pas gerust als test bovenstaande code aan als volgt:
```python
GPIO26.irq(button_isr,trigger=Pin.IRQ_FALLING,hard=True)
```

### Analoog

## Outputs

### Digitaal

Om het voorbeeld Blink werkende te krijgen is dit gedeelte al summier opgenomen. Voor iedere processor zijn er additionale klasses opgenomen bij Python (daarom dat ook de juiste versie van MicroPython moet geïnstalleerd worden). Voor de ESP32 beschik je als gebruik over de klasse `machine.Pin`. Deze bevat volgende methoden die bruikbaar zijn voor een output:
1) `.on()` maakt de pin hoog. _Opgelet, via `machine.Signal` kan de werking van de pin geïnverteerd worden. Een `.on()` zal dan resulteren in een pin die laag gemaakt wordt._
2) `.off()` maakt de pin laag. _Ook hier geldt dezelfde opmerking._
3) `.value(1)` of `.value(0)` maakt de pin respectievelijk hoog of laag.
4) 

### PWM